<a href="https://colab.research.google.com/github/harshvs4/Coding_Drill_Down_MNIST/blob/main/Exp7_GAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR,OneCycleLR

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomRotation((-6.0, 6.0), fill=(1,)),                
                                       transforms.RandomAffine(degrees=7, shear=10, translate=(0.1, 0.1), scale=(0.8, 1.2)),
                                       transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.40, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [ ]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [ ]:
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=2, pin_memory=True) if use_cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 4, 3, padding = 0, bias = False),  # (1*26*26), (4*26*26), RF - 3*3
        nn.ReLU(),
        nn.BatchNorm2d(4),
        #nn.Dropout(0.05),

        nn.Conv2d(4, 8, 3, padding = 0, bias = False), #(4*24*24), (8*24*24), RF - 5*5
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.Dropout(0.05)
    )

    #Transition block
    self.trans1 = nn.Sequential(
        nn.MaxPool2d(2, 2) , #(8*12*12), RF - 10*10
        nn.Conv2d(8, 10, 1, padding = 0, bias = False),  #(8*12*12), (16*12*12), RF - 10*10
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Dropout(0.05)
    )

    self.conv2 = nn.Sequential(
        nn.Conv2d(10, 16, 3, padding = 0, bias = False), #(10*10*10), (16*10*10), RF - 12*12
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(0.05),

        nn.Conv2d(16, 16, 3, padding = 0, bias = False), #(16*8*8), (16*8*8), RF - 14*14
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(0.05),

        nn.Conv2d(16, 18, 3, padding = 0, bias = False), #(16*8*8),(18*8*8), RF - 14*14 
        nn.ReLU(),
        nn.BatchNorm2d(18),
        nn.Dropout(0.05)
    )

    #Output block
    self.avgpool2d = nn.AvgPool2d(6,6) #RF - 28*28
    self.conv3 = nn.Sequential(
        nn.Conv2d(18, 16, 1, padding = 0, bias = False), #(18*6*6), (16*6*6), RF - 28*28
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(0.05)
    )
    self.conv4 = nn.Conv2d(16, 10, 1, padding = 0, bias = False)
    

  def forward(self, x):
       x = self.conv1(x)
       x = self.trans1(x)
       x = self.conv2(x)
       x = self.avgpool2d(x)
       x = self.conv3(x)
       x = self.conv4(x)
       x = x.view(-1,10)
       return F.log_softmax(x,dim=1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              36
              ReLU-2            [-1, 4, 26, 26]               0
       BatchNorm2d-3            [-1, 4, 26, 26]               8
            Conv2d-4            [-1, 8, 24, 24]             288
              ReLU-5            [-1, 8, 24, 24]               0
       BatchNorm2d-6            [-1, 8, 24, 24]              16
           Dropout-7            [-1, 8, 24, 24]               0
         MaxPool2d-8            [-1, 8, 12, 12]               0
            Conv2d-9           [-1, 10, 12, 12]              80
             ReLU-10           [-1, 10, 12, 12]               0
      BatchNorm2d-11           [-1, 10, 12, 12]              20
          Dropout-12           [-1, 10, 12, 12]               0

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = OneCycleLR(optimizer, max_lr=0.015,epochs=15,steps_per_epoch=len(train_loader))

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 1


Loss=0.4785846173763275 Batch_id=468 Accuracy=63.62: 100%|██████████| 469/469 [00:35<00:00, 13.21it/s]



Test set: Average loss: 0.2167, Accuracy: 9474/10000 (94.74%)

EPOCH: 2


Loss=0.2947288155555725 Batch_id=468 Accuracy=92.68: 100%|██████████| 469/469 [00:35<00:00, 13.22it/s]



Test set: Average loss: 0.0602, Accuracy: 9821/10000 (98.21%)

EPOCH: 3


...


Test set: Average loss: 0.0172, Accuracy: 9941/10000 (99.41%)

EPOCH: 14


Loss=0.03951990231871605 Batch_id=468 Accuracy=98.27: 100%|██████████| 469/469 [00:34<00:00, 13.53it/s]



Test set: Average loss: 0.0167, Accuracy: 9942/10000 (99.42%)

EPOCH: 15


Loss=0.01646753028035164 Batch_id=468 Accuracy=98.31: 100%|██████████| 469/469 [00:34<00:00, 13.41it/s]



Test set: Average loss: 0.0159, Accuracy: 9940/10000 (99.40%)

